In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

%matplotlib inline 


# Simple distribution visualization of the comment_score variable

Let's look at the the only 2 numerical variables in our dataset. 

Submission upvotes are number of upvotes or likes that a post containing comments has. It mainly conveys how the title of the post or the discussion is attractive to the user to participate in. Basically one can treat the number of upvotes as indicator how good online conversation topic it is, with higher values being more interesting. 

Score variable is equal to the difference between upvotes and downvotes. It usually conveys information how useful information does the comment give to the reader or weather other users agree with the comment or disagree.

Below we can see summary statistics of these variables for men and women datasets